# Introduction to AutoGluon

AutoGluon is an open-source library designed to simplify the process of machine learning by automating the model selection and training process. It’s particularly useful for tabular data, and allows you to train high-quality models with minimal effort and code.

**Key features of AutoGluon:**
- **AutoML for Tabular Data**: AutoGluon automatically selects and trains a variety of models (like Random Forests, XGBoost, Neural Networks, etc.) to find the best-performing model for your dataset.
- **Ensemble Methods**: AutoGluon combines different models through ensembling techniques to boost prediction accuracy.
- **Easy-to-Use API**: With only a few lines of code, you can build powerful machine learning models.
- **Hyperparameter Optimization**: AutoGluon automates the process of hyperparameter tuning, helping you find the best parameters for your models.
- **Supports Multiple Task Types**: You can use AutoGluon for classification, regression, and other tasks with minimal configuration.

AutoGluon is an excellent choice for users who want to quickly build predictive models without needing to fine-tune machine learning algorithms manually.


In [2]:
# Install AutoGluon library for tabular data prediction
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple vers

In [1]:
# Install the Kaggle package to enable Kaggle API functionality
!pip install kaggle

In [2]:
# Upload the kaggle.json file to Colab
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"subhashpolisetti347","key":"0590675aeb1ac3bbbb4f3b6a4ac2e351"}'}

In [3]:
# Create the Kaggle directory if it doesn't exist
!mkdir -p ~/.kaggle

# Move the kaggle.json file to this directory
!mv kaggle.json ~/.kaggle/

# Set the required permissions for the file
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download the IEEE-CIS Fraud Detection dataset from Kaggle
!kaggle competitions download -c ieee-fraud-detection

# Unzip the dataset
!unzip ieee-fraud-detection.zip


 95% 112M/118M [00:00<00:00, 153MB/s] 
100% 118M/118M [00:00<00:00, 154MB/s]
Archive:  ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


In [5]:
import pandas as pd

# Directory where the unzipped CSV files are located
directory = '/content/'

# Load the transaction and identity datasets
train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

# Merge the two datasets on 'TransactionID'
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

# Check the first few rows to ensure data is loaded correctly
train_data.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [6]:
from autogluon.tabular import TabularPredictor

# Define the target label and evaluation metric
label = 'isFraud'
eval_metric = 'roc_auc'

# Define the save path for AutoGluon models
save_path = '/content/AutoGluonModels/'

# Train the model with AutoGluon
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='good_quality', time_limit=3600, excluded_model_types=['NN', 'STACKER'], keep_only_best=True
)

# Print the summary of the fit process
results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          96
GPU Count:          0
Memory Avail:       324.49 GB / 334.56 GB (97.0%)
Disk Space Avail:   197.25 GB / 225.33 GB (87.5%)
Presets specified: ['good_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True,
 'excluded_model_types': ['NN', 'STACKER'],
 'keep_only_best': True,
 'num_bag_sets': 1,
 'refit_full': True,
 'save_bag_folds': False,
 'set_best_to_refit_full': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True

(_ray_fit pid=16505) [50]	valid_set's binary_logloss: 0.0964283
(_ray_fit pid=16508) [100]	valid_set's binary_logloss: 0.0852749 [repeated 10x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(_ray_fit pid=16504) [150]	valid_set's binary_logloss: 0.0826913 [repeated 12x across cluster]
(_ray_fit pid=16507) [250]	valid_set's binary_logloss: 0.077055 [repeated 12x across cluster]
(_ray_fit pid=16509) [300]	valid_set's binary_logloss: 0.0745371 [repeated 11x across cluster]
(_ray_fit pid=16505) [400]	valid_set's binary_logloss: 0.072224 [repeated 13x across cluster]
(_ray_fit pid=16506) [500]	valid_set's binary_logloss: 0.0668289 [repeated 14x across cluster]
(_ray_fit pid=16508) [600]	valid_set's binary_logloss: 0.0649414 [repeated 14x across cluster]
(_ray_fit pid=16510) [600]	valid_set's binary_logloss: 0.065767 [repeated 13x 

(_dystack pid=8788) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/utils/oof.pkl
(_dystack pid=8788) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBMXT_BAG_L1/model.pkl
(_dystack pid=8788) 	0.9698	 = Validation score   (roc_auc)
(_dystack pid=8788) 	461.84s	 = Training   runtime
(_dystack pid=8788) 	230.94s	 = Validation runtime
(_dystack pid=8788) 	284.1	 = Inference  throughput (rows/s | 65616 batch size)
(_dystack pid=8788) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=8788) Fitting model: LightGBM_BAG_L1 ... Training model for up to 73.25s of the 360.69s of remaining time.
(_dystack pid=8788) 	Fitting LightGBM_BAG_L1 with 'num_gpus': 0, 'num_cpus': 96
(_dystack pid=8788) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBM_BAG_L1/utils/model_template.pkl
(_dystack pid=8788) Loading: /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBM_BAG_L1/utils/model_template.

(_ray_fit pid=19462) [50]	valid_set's binary_logloss: 0.0870287 [repeated 4x across cluster]
(_ray_fit pid=19462) [100]	valid_set's binary_logloss: 0.078415 [repeated 10x across cluster]
(_ray_fit pid=19464) [150]	valid_set's binary_logloss: 0.0745762 [repeated 9x across cluster]
(_ray_fit pid=19463) [250]	valid_set's binary_logloss: 0.0718996 [repeated 11x across cluster]
(_ray_fit pid=19465) [250]	valid_set's binary_logloss: 0.0711994 [repeated 15x across cluster]
(_ray_fit pid=19459) [450]	valid_set's binary_logloss: 0.0638379 [repeated 15x across cluster]
(_ray_fit pid=19466) [500]	valid_set's binary_logloss: 0.0624308 [repeated 13x across cluster]
(_ray_fit pid=19463) [600]	valid_set's binary_logloss: 0.062946 [repeated 11x across cluster]


(_dystack pid=8788) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBM_BAG_L1/utils/oof.pkl
(_dystack pid=8788) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/LightGBM_BAG_L1/model.pkl
(_dystack pid=8788) 	0.9523	 = Validation score   (roc_auc)
(_dystack pid=8788) 	60.66s	 = Training   runtime
(_dystack pid=8788) 	9.71s	 = Validation runtime
(_dystack pid=8788) 	6759.4	 = Inference  throughput (rows/s | 65616 batch size)
(_dystack pid=8788) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/trainer.pkl
(_dystack pid=8788) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 3.82s of the 291.26s of remaining time.
(_dystack pid=8788) 	Fitting RandomForestGini_BAG_L1 with 'num_gpus': 0, 'num_cpus': 96
(_dystack pid=8788) Saving /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/RandomForestGini_BAG_L1/utils/model_template.pkl
(_dystack pid=8788) Loading: /content/AutoGluonModels/ds_sub_fit/sub_fit_ho/models/RandomForestGini_BAG

...

Saving /content/AutoGluonModels/models/CatBoost_BAG_L2_FULL/model.pkl
	54.3s	 = Training   runtime
Saving /content/AutoGluonModels/models/trainer.pkl
Loading: /content/AutoGluonModels/models/WeightedEnsemble_L3/model.pkl
Fitting model: WeightedEnsemble_L3_FULL | Skipping fit via cloning parent ...
	Ensemble Weights: {'LightGBMXT_BAG_L2': 0.36, 'LightGBM_BAG_L2': 0.32, 'CatBoost_BAG_L2': 0.28, 'RandomForestEntr_BAG_L2': 0.04}
	25.9s	 = Training   runtime
Saving /content/AutoGluonModels/models/WeightedEnsemble_L3_FULL/model.pkl
Saving /content/AutoGluonModels/models/trainer.pkl


217:	learn: 0.0361477	total: 49.7s	remaining: 0us


Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl
Saving /content/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/LightGBM_BAG_L1_FULL/model.pkl
Saving /content/AutoGluonModels/models/LightGBM_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestGini_BAG_L1_FULL/model.pkl
Saving /content/AutoGluonModels/models/RandomForestGini_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestEntr_BAG_L1_FULL/model.pkl
Saving /content/AutoGluonModels/models/RandomForestEntr_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/CatBoost_BAG_L1_FULL/model.pkl
Saving /content/AutoGluonModels/models/CatBoost_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L2_FULL/model.pkl
Saving /content/AutoGluonModels/models/LightGBMXT_BAG_L2_FULL/model.pkl
Loading: /content/AutoGluonModels/models/LightGBM_BAG_L2_FULL/model.pkl
Saving /content/AutoGluonModels/mode

*** Summary of fit() ***
Estimated performance of each model:
                          model score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  RandomForestGini_BAG_L1_FULL      None     roc_auc     125.710804   37.550590              125.710804          37.550590            1       True          3
1  RandomForestEntr_BAG_L1_FULL      None     roc_auc     127.032806   33.069827              127.032806          33.069827            1       True          4
2      WeightedEnsemble_L3_FULL      None     roc_auc            NaN  591.306964                     NaN          25.897851            3       True         10
3  RandomForestEntr_BAG_L2_FULL      None     roc_auc            NaN  472.592563              128.499431          33.567673            2       True          8
4          LightGBM_BAG_L2_FULL      None     roc_auc            NaN  453.212598                     NaN          14.187708            2       

/usr/local/lib/python3.10/dist-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [9]:
# Display the column names of the training data
print("Training Data Columns:")
print(train_data.columns)



Training Data Columns:
Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)


In [10]:
# Replace hyphens with underscores in train_identity column names to match train_transaction
train_identity.columns = train_identity.columns.str.replace('-', '_')



In [11]:
# Merge the training datasets on 'TransactionID'
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')


In [12]:
# Display the first few rows of the training data
print("First few rows of the training data:")
print(train_data.head())

First few rows of the training data:
   TransactionID  isFraud  TransactionDT  TransactionAmt ProductCD  card1  \
0        2987000        0          86400            68.5         W  13926   
1        2987001        0          86401            29.0         W   2755   
2        2987002        0          86469            59.0         W   4663   
3        2987003        0          86499            50.0         W  18132   
4        2987004        0          86506            50.0         H   4497   

   card2  card3       card4  card5  ...                id_31  id_32  \
0    NaN  150.0    discover  142.0  ...                  NaN    NaN   
1  404.0  150.0  mastercard  102.0  ...                  NaN    NaN   
2  490.0  150.0        visa  166.0  ...                  NaN    NaN   
3  567.0  150.0  mastercard  117.0  ...                  NaN    NaN   
4  514.0  150.0  mastercard  102.0  ...  samsung browser 6.2   32.0   

       id_33           id_34  id_35 id_36 id_37  id_38  DeviceType  \
0  

In [14]:
# Make predictions on the test data
# Note: Since 'isFraud' is not in test_data, ensure it is removed from the features if present
if 'isFraud' in train_data.columns:
    test_data = train_data.drop(columns=['isFraud'])

In [15]:
# Predict the probability of the positive class (fraudulent transactions)
y_pred_proba = predictor.predict_proba(test_data, as_multiclass=False)

Loading: /content/AutoGluonModels/models/CatBoost_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/LightGBM_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestEntr_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestGini_BAG_L1_FULL/model.pkl
Loading: /content/AutoGluonModels/models/CatBoost_BAG_L2_FULL/model.pkl
Loading: /content/AutoGluonModels/models/LightGBMXT_BAG_L2_FULL/model.pkl
Loading: /content/AutoGluonModels/models/LightGBM_BAG_L2_FULL/model.pkl
Loading: /content/AutoGluonModels/models/RandomForestEntr_BAG_L2_FULL/model.pkl
Loading: /content/AutoGluonModels/models/WeightedEnsemble_L3_FULL/model.pkl


In [16]:
# Load the sample submission file
submission = pd.read_csv(directory + 'sample_submission.csv')

In [18]:
# Display the first few rows of the submission file
print("First few rows of the submission file:")
print(submission.head())


First few rows of the submission file:
   TransactionID  isFraud
0        3663549      0.5
1        3663550      0.5
2        3663551      0.5
3        3663552      0.5
4        3663553      0.5


In [19]:
# Save the submission file
submission.to_csv(directory + 'my_submission.csv', index=False)

print("\nSubmission file saved to:", directory + 'my_submission.csv')



Submission file saved to: /content/my_submission.csv
